In [1]:
import torch
import torch.nn as nn
import numpy as np
from model import get_vit
import args
from data import get_cifar10_dataloader
from train import eval_model

d:\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = r'D:\d-storage\output\pat-0.5.pt'
trainloader = get_cifar10_dataloader(train=True)
testloader = get_cifar10_dataloader(train=False)
mqvit = get_vit(args.MQVIT, path)

_IncompatibleKeys(missing_keys=['vit.transformer.encoder_layers.0.attn.query.activation_range', 'vit.transformer.encoder_layers.0.attn.query.weight_range', 'vit.transformer.encoder_layers.0.attn.key.activation_range', 'vit.transformer.encoder_layers.0.attn.key.weight_range', 'vit.transformer.encoder_layers.0.attn.value.activation_range', 'vit.transformer.encoder_layers.0.attn.value.weight_range', 'vit.transformer.encoder_layers.0.attn.out.activation_range', 'vit.transformer.encoder_layers.0.attn.out.weight_range', 'vit.transformer.encoder_layers.0.mlp.fc1.activation_range', 'vit.transformer.encoder_layers.0.mlp.fc1.weight_range', 'vit.transformer.encoder_layers.0.mlp.fc2.activation_range', 'vit.transformer.encoder_layers.0.mlp.fc2.weight_range', 'vit.transformer.encoder_layers.1.attn.query.activation_range', 'vit.transformer.encoder_layers.1.attn.query.weight_range', 'vit.transformer.encoder_layers.1.attn.key.activation_range', 'vit.transformer.encoder_layers.1.attn.key.weight_range', 

In [3]:
from mmsa import mix_prune, get_mask_idx
from quant_utils import set_mixed_precision, get_quantizable_idx, mix_weight_act_strategy

In [4]:
q_idx = get_quantizable_idx(mqvit)
set_mixed_precision(mqvit, q_idx, strategy=mix_weight_act_strategy([8]*len(q_idx), [8]*len(q_idx)))

In [5]:
p_idx= get_mask_idx(mqvit)
mix_prune(mqvit, p_idx, [.75]*len(p_idx))

In [6]:
mqvit

CAFIA_Transformer(
  (vit): VisionTransformer(
    (embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (transformer): Encoder(
      (pos_embedding): PositionEmbs(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder_layers): ModuleList(
        (0): EncoderBlock(
          (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MaskedSelfAttention(
            (query): QLinearGeneral(in_dim=(768,), feat_dim=(12, 64), w_bit=8, a_bit=8, half wave, tensor wise)
            (key): QLinearGeneral(in_dim=(768,), feat_dim=(12, 64), w_bit=8, a_bit=8, half wave, tensor wise)
            (value): QLinearGeneral(in_dim=(768,), feat_dim=(12, 64), w_bit=8, a_bit=8, half wave, tensor wise)
            (out): QLinearGeneral(in_dim=(12, 64), feat_dim=(768,), w_bit=8, a_bit=8, half wave, tensor wise)
            (mask): LearnableMask(p=1.0, fixed=True, pruned dim=3)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (